# Spatial Join to HQTA

* Which HQTA file to use? Can use polygons first, and filter out the ferry and rail ones.
* Seems like we don't have that many facilities to retrofit anyway.
* Don't overcomplicate and think about whether it's near a bus's origin or destination.
* Start with overall, within a district, a map showing both facility locations (points) and HQTA polygons.
* In DTLA, that Caltrans building could be retrofitted, and the corridors it would serve are numerous.

Folium ideas:
* Use GeoJSON works, can use multiple layers function, but can't control marker
* Using Marker or CircleMarker gets it to look nicer, but output is way larger for html
* https://github.com/python-visualization/folium/issues/1059
* https://github.com/python-visualization/folium/pull/957
* look at this script: https://github.com/jtbaker/folium/blob/83745caa3613aede3d435909a325e13f9c50dde9/folium/features.py
* ex: https://github.com/jtbaker/folium/blob/83745caa3613aede3d435909a325e13f9c50dde9/examples/GeoJSONMarker.ipynb


In [ ]:
import geopandas as gpd
import intake
import pandas as pd

import utils
from shared_utils import geography_utils

catalog = intake.open_catalog("./*.yml")

In [ ]:
facilities = catalog.tier1_facilities_processed.read()

In [ ]:
hqta = catalog.hqta_shapes.read()

exclude_me = ["major_stop_ferry", 
           "major_stop_rail",
          ]

hqta = hqta[~hqta.hqta_type.isin(exclude_me)]

In [ ]:
hqta_cols = ['calitp_itp_id_primary', 'agency_name_primary', 'hqta_type',
       'calitp_itp_id_secondary', 'agency_name_secondary',]
hqta2 = hqta.dissolve(by=hqta_cols).reset_index()

In [ ]:
# spatial join
gdf = gpd.sjoin(
    facilities.to_crs(geography_utils.WGS84),
    hqta2.to_crs(geography_utils.WGS84),
    how = "inner",
    predicate = "intersects",
).drop(columns = "index_right").drop_duplicates()

In [ ]:
gdf.sheet_uuid.nunique()

In [ ]:
# Which locations
facility_cols = ["sqft", "category", 
                 "facility_name", "facility_type",
                 "address_arcgis_clean", 
                 "county_name", "district",
                ]
facilities = gdf[["sheet_uuid", "geometry"] + facility_cols].drop_duplicates()

# Just draw a small buffer around the point for now
# Then use function to plot multiple polygon layers
# Tweak that map_utils multiple layers function to take points?
'''
facilities = facilities.assign(
    longitude = facilities.geometry.x,
    latitude = facilities.geometry.y,
    geometry = (facilities.to_crs(geography_utils.CA_StatePlane).buffer(25)
                .to_crs(geography_utils.WGS84)
               )
)
'''

In [ ]:
# Which HQTA corridors (polygon geom)
hqta_cols = [
    'calitp_itp_id_primary',
    'agency_name_primary', 
    'hqta_type', 
    'calitp_itp_id_secondary',
    'agency_name_secondary'
]

hqta_corr = (gdf[["sheet_uuid"] + facility_cols + hqta_cols].drop_duplicates()
             .merge(hqta2,
                    on = hqta_cols,
                    how = "inner"
                   )
            )

hqta_corr = gpd.GeoDataFrame(hqta_corr).to_crs(geography_utils.WGS84)

In [ ]:
# A lot of these are in D4, D7, and a bunch that's HQ (subset of D3 geographically)
# Can do statewide map, but district stats
# For district, show breakdown of facility category / list of facility names

In [ ]:
facilities.district.value_counts()

In [ ]:
facilities.category.value_counts()

In [ ]:
keep_uuid = "6dd5b375-2a3d-4ebb-8dea-26518e4a9cd9"
hqta_corr = hqta_corr[hqta_corr.sheet_uuid == keep_uuid]
facilities = facilities[facilities.sheet_uuid==keep_uuid]

## Folium Map

Adjust `map_utils` to take a polygon and a point as the 2 layers.

In [ ]:
import inspect

import folium
from branca.element import Figure
from folium.features import GeoJsonPopup, GeoJsonTooltip
from folium.plugins import FloatImage

import branca


from shared_utils import map_utils

In [ ]:
TOOLTIP_KWARGS = {
    "min_width": 50,
    "max_width": 100,
    "font_size": "12px",
}


def format_folium_popup(popup_dict):
    popup = GeoJsonPopup(
        fields=list(popup_dict.keys()),
        aliases=list(popup_dict.values()),
        # localize=True,
        labels=True,
        style="background-color: light_gray;",
        min_width=TOOLTIP_KWARGS["min_width"],
        max_width=TOOLTIP_KWARGS["max_width"],
    )
    return popup


def format_folium_tooltip(tooltip_dict):
    tooltip = GeoJsonTooltip(
        fields=list(tooltip_dict.keys()),
        aliases=list(tooltip_dict.values()),
        # localize = True sets the commas for numbers, but zipcode should be displayed as string
        # localize=True,
        sticky=False,
        labels=True,
        style=f"""
            background-color: "gray";
            border: 0px #FFFFFF;
            border-radius: 0px;
            box-shadow: 0px;
            font-size: {TOOLTIP_KWARGS["font_size"]};
        """,
        min_width=TOOLTIP_KWARGS["min_width"],
        max_width=TOOLTIP_KWARGS["max_width"],
    )
    return tooltip



In [ ]:
def make_folium_multiple_layers_map(
    LAYERS_DICT,
    fig_width,
    fig_height,
    zoom=map_utils.REGION_CENTROIDS["CA"]["zoom"],
    centroid=map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title="Chart Title",
    legend_dict={"legend_url": "", "legend_bottom": 85, "legend_left": 5},
    **kwargs,
):
    """
    Parameters:
    LAYERS_DICT: dict. Can contain as many other polygon layers as needed.
        Must contain the following key-value pairs
        key: name of the layer
        value: dict with relevant info for plotting the layer
                df, plot_col, popup_dict, tooltip_dict, colorscale
        LAYERS_DICT = {
            "layer_name1": {"df": geopandas.GeoDataFrame,
                "plot_col": str,
                "popup_dict": dict,
                "tooltip_dict": dict,
                "colorscale": branca.colormap element
            },
            "layer_name2": {"df": geopandas.GeoDataFrame,
                "plot_col": str,
                "popup_dict": dict,
                "tooltip_dict": dict,
                "colorscale": branca.colormap element
            },
        }
    fig_width: int. Ex: 500
    fig_height: int. Ex: 500
    zoom: int.
    centroid: list, of the format [latitude, longitude]
    title: str.
    legend_dict: dict
        legend_dict = {
            "legend_url": str
                        GitHub url to the image of the legend manually created
                        Ex:  ('https://raw.githubusercontent.com/cal-itp/data-analyses/'
                                'more-highways/bus_service_increase/'
                                'img/legend_intersecting_parallel.png'
                        )
            "legend_bottom": int
                            value between 0-100, relative to the bottom edge of figure
            "legend_left": int
                            value between 0-100, relative to the left edge of figure
        }
    **kwargs: any other keyword arguments that can passed into existing folium functions
            that are used in this function
    """

    # Pass more kwargs through various sub-functions
    # https://stackoverflow.com/questions/26534134/python-pass-different-kwargs-to-multiple-functions
    fig_args = [k for k, v in inspect.signature(Figure).parameters.items()]
    fig_dict = {k: kwargs.pop(k) for k in dict(kwargs) if k in fig_args}

    fig = Figure(width=fig_width, height=fig_height, **fig_dict)

    map_args = [k for k, v in inspect.signature(folium.Map).parameters.items()]
    map_dict = {k: kwargs.pop(k) for k in dict(kwargs) if k in map_args}

    m = folium.Map(
        location=centroid,
        tiles="cartodbpositron",
        zoom_start=zoom,
        width=fig_width,
        height=fig_height,
        **map_dict,
    )

    title_html = f"""
         <h3 align="center" style="font-size:20px"><b>{title}</b></h3>
         """

    fig.get_root().html.add_child(folium.Element(title_html))

    # Define function that can theoretically pop out as many polygon layers as needed
    def get_layer(
        df, plot_col, popup_dict, tooltip_dict, colorscale, layer_name, **kwargs
    ):

        popup = format_folium_popup(popup_dict)
        tooltip = format_folium_tooltip(tooltip_dict)

        # https://medium.com/analytics-vidhya/create-and-visualize-choropleth-map-with-folium-269d3fd12fa0
        geojson_args = [
            k for k, v in inspect.signature(folium.GeoJson).parameters.items()
        ]
        geojson_dict = {k: kwargs.pop(k) for k in dict(kwargs) if k in geojson_args}
        
        g = folium.GeoJson(
            df,
            style_function=lambda x: {
                "fillColor": colorscale(x["properties"][plot_col])
                if x["properties"][plot_col] is not None
                else "gray",
                "color": "#FFFFFF",
                "fillOpacity": 0.8,
                "weight": 0.2,
            },
            tooltip=tooltip,
            popup=popup,
            name=layer_name,
            **geojson_dict,
        )

        return g

    # Now, loop through the keys in the LAYERS_DICT,
    # Unpack the dictionary associated with each layer,
    # Then attach that layer to the Map element
    for key, nested_dict in LAYERS_DICT.items():
        # key: layer name or layer number
        # value: dictionary of all the components associated with folium layer
        d = nested_dict
        
        layer = get_layer(
            df=d["df"],
            plot_col=d["plot_col"],
            popup_dict=d["popup_dict"],
            tooltip_dict=d["tooltip_dict"],
            colorscale=d["colorscale"],
            layer_name=key,
        )
        layer.add_to(m)

    # Legend doesn't show up with multiple layers
    # One way around, create the colorscale(s) as one image and save it
    # Then, insert that legend as a URL to be an image
    # I think legend_bottom and legend_left numbers must be 0-100?
    # Going even 95 pushes it to the top edge of the figure
    FloatImage(
        legend_dict["legend_url"],
        legend_dict["legend_bottom"],
        legend_dict["legend_left"],
    ).add_to(m)

    folium.LayerControl("topright", collapsed=False).add_to(m)

    # Now, attach everything to Figure
    fig.add_child(m)

    return fig

In [ ]:
hqta_popup = {
    "hqta_type": "HQTA Type",
    "agency_name_primary": "Primary Agency",
    "agency_name_secondary": "Secondary Agency",
}

color_hqta = branca.colormap.step.Accent_07

facilities_popup = {
    "facility_name": "Name",
    "address_arcgis_clean": "Address" ,
    "category": "Facility Category",
    "facility_type": "Facility Type"
}

color_facilities = branca.colormap.step.viridis

LAYERS_DICT = {
    "hqta": {
        "df": hqta_corr,
        "plot_col": "sqft",
        "popup_dict": hqta_popup,
        "tooltip_dict": hqta_popup,
        "colorscale": color_hqta,
    },
    "facilities": {
        "df": facilities,
        "plot_col": "sqft",
        "popup_dict": facilities_popup,
        "tooltip_dict": facilities_popup,
        "colorscale": color_facilities,
        "marker": folium.Circle(radius=4, fill_color="black", 
                             fill_opacity=0.9, 
                             color="black", weight=2),
        }
    }

In [ ]:
fig_width = 500
fig_height = 800
zoom = 9
centroid = map_utils.REGION_CENTROIDS["Los Angeles"]["centroid"]
title = "Map Title"
legend_name = "Legend"

In [ ]:
make_folium_multiple_layers_map(
    LAYERS_DICT,
    fig_width, 
    fig_height,
    zoom,
    centroid,
    title,
)

In [ ]:
point_layer = folium.GeoJson(
    facilities,
    #name="Subway Stations",
    marker=folium.Circle(radius=4, fill_color="yellow", fill_opacity=0.9, color="black", weight=2),
    style_function=lambda x: {
        "fillColor": color_facilities(x["properties"]["sqft"]),
        #"radius": (x['properties']["sqft"])*30,
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
    #zoom_on_click=True,
)

In [ ]:
fig = Figure(width=fig_width, height=fig_height)


m = folium.Map(
    location=centroid,
    tiles="cartodbpositron",
    zoom_start=zoom,
    width=fig_width,
    height=fig_height,
)

title_html = f"""
         <h3 align="center" style="font-size:20px"><b>{title}</b></h3>
         """

fig.get_root().html.add_child(folium.Element(title_html))


# https://medium.com/analytics-vidhya/create-and-visualize-choropleth-map-with-folium-269d3fd12fa0

folium.GeoJson(
    hqta_corr,
    style_function=lambda x: {
        "fillColor": color_hqta(x["properties"]["sqft"])
        if x["properties"]["sqft"] is not None
        else "gray",
        "color": "#FFFFFF",
        "fillOpacity": 0.8,
        "weight": 0.2,
    },
    name={legend_name},
).add_to(m)

fig.add_child(m)

point_layer.add_to(m)

fig.add_child(m)